In [35]:
from sklearn.naive_bayes import BernoulliNB
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk import download
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# download("stopwords")
# download("wordnet")

path = "/home/dmitry/Downloads/SMSSpamCollection"
df = pd.read_table(path, names=['label', 'text'])

In [38]:
# clear_df = df["text"].apply(lambda x: x.lower()).apply(lambda y: " ".join(re.findall(r"\w+", y)))


In [36]:
def prepare_df(row):
    row = re.findall(r"\w+", row.lower())
    row = map(lambda x: lemmatizer.lemmatize(x, "v"), row)
    return " ".join(filter(lambda x: x not in stopwords.words("english"), row))

In [39]:
clean_df = df["text"].apply(prepare_df)

In [49]:
uniq_words = set()
list(map(lambda x: uniq_words.update(x.split()), clean_df));


In [56]:
space = len(uniq_words)

In [50]:
word2index = {}
for index, word in enumerate(uniq_words):
    word2index[word] = index

In [51]:
def conver_word2index(row):
    return [word2index[i] for i in row.split()]

In [53]:
clean_df_num = clean_df.apply(conver_word2index)

In [57]:
def add_space(row):
    while len(row) < 78:
        row.append(space)
    return row

In [58]:
clean_df_num_equal = clean_df_num.apply(add_space)
clean_df_num_equal

0       [5781, 2575, 4693, 1750, 6126, 1722, 4806, 652...
1       [6887, 490, 2782, 6086, 4840, 5161, 7639, 7639...
2       [1801, 1787, 3908, 5893, 6975, 5267, 1173, 178...
3       [4840, 858, 2072, 398, 4567, 4840, 288, 7394, ...
4       [5074, 2445, 5781, 2221, 7556, 6391, 3695, 763...
                              ...                        
5567    [2097, 3414, 4979, 3908, 2995, 4840, 4840, 526...
5568    [3037, 3294, 5781, 6075, 1457, 2233, 7639, 763...
5569    [4798, 5979, 540, 7639, 7639, 7639, 7639, 7639...
5570    [2614, 4483, 1283, 2543, 4461, 160, 4950, 4096...
5571    [5841, 655, 2497, 7639, 7639, 7639, 7639, 7639...
Name: text, Length: 5572, dtype: object

In [59]:
df["label"] = df["label"].apply(lambda x: 1 if x == "spam" else 0)

In [66]:
from sklearn.model_selection import train_test_split
import numpy as np
x_train, x_test, y_train, y_test = train_test_split(clean_df_num_equal, df["label"])

In [67]:
x_train = np.array([np.array(i) for i in x_train])
x_test = np.array([np.array(i) for i in x_test])

In [68]:
model = BernoulliNB()
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.8657573582196698

---

In [86]:
df_backwords = df["text"].apply(prepare_df)
df_backwords_l = df["label"].apply(lambda x: 1 if x == "spam" else 0)

In [78]:
uniq_words = set()
list(map(lambda x: uniq_words.update(x.split()), df_backwords));

space = len(uniq_words)

word2index = {}
for index, word in enumerate(uniq_words, 1):
    word2index[word] = index
length_row = len(uniq_words)


In [79]:
def encode_row(row):
    out = np.zeros(length_row + 1)
    row = row.split()
    for word in row:
        index = word2index[word]
        out[index] = row.count(word)
    return out
df_bw_encoded = df_backwords.apply(encode_row)

In [92]:
x_train, x_test, y_train, y_test = train_test_split(df_backwords, df_backwords_l)
x_train = np.array([np.array(i) for i in x_train])
x_test = np.array([np.array(i) for i in x_test])

# model = BernoulliNB()
# model.fit(x_train, y_train)
# model.score(x_test, y_test)

In [95]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.datasets import make_classification
